In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.python.ops import rnn, rnn_cell

In [2]:
#Defining some hyper-params
n_input = 2     # this is the parameter for input_size in the basic LSTM cell
n_hidden = 40    # num_units and input_size will be the same
n_out = 1
batch_size = 100
n_steps = 55
min_sequence_len = 45
max_sequence_len = n_steps

In [3]:
def gen_data(min_length=50, max_length=55, n_batch=5):    
    '''
    Generate a batch of sequences for the "add" task, e.g. the target for the following
    | 0.5 | 0.7 | 0.3 | 0.1 | 0.2 | ... | 0.5 | 0.9 | ... | 0.8 | 0.2 |
    |  0  |  0  |  1  |  0  |  0  |     |  0  |  1  |     |  0  |  0  |
    would be 0.3 + .9 = 1.2. 
    
    Parameters
    ----------
    min_length : int
        Minimum sequence length.
    max_length : int
        Maximum sequence length.
    n_batch : int
        Number of samples (sequences) in the batch.
    Returns
    -------
    X : Input to the network, of shape (n_batch, max_length, 2), where the last
        dimensions corresponds to the two sequences shown above.
    y : Correct output for each sample, of shape (n_batch,)
    '''
    
    X = np.concatenate([np.random.uniform(size=(n_batch, max_length, 1)),
                        np.zeros((n_batch, max_length, 1))], axis=-1)
    y = np.zeros((n_batch,))
    # Compute masks and correct values
    for n in range(n_batch):
        # Randomly choose the sequence length
        length = np.random.randint(min_length, max_length)
        # Zero out X after the end of the sequence
        X[n, length:, 0] = 0
        # Set the second dimension to 1 at the indices to add
        X[n, np.random.randint(length/2-1), 1] = 1
        X[n, np.random.randint(length/2, length), 1] = 1
        # Multiply and sum the dimensions of X to get the target value
        y[n] = np.sum(X[n, :, 0]*X[n, :, 1])
    return (X,y)    

In [4]:
# tf Graph input
x = tf.placeholder("float", [None, n_steps, n_input])
y = tf.placeholder("float", [None])

def RNN(x):
    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, n_steps, n_input)
    # Required shape: 'n_steps' tensors list of shape (batch_size, n_input)
    
    # Permuting batch_size and n_steps
    x = tf.transpose(x, [1, 0, 2])
    # Reshaping to (n_steps*batch_size, n_input)
    x = tf.reshape(x, [-1, n_input])
    # Split to get a list of 'n_steps' tensors of shape (batch_size, n_input)
    x = tf.split(0, n_steps, x)

    # Define an lstm cell with tensorflow
    lstm_cell = rnn_cell.BasicLSTMCell(n_hidden, forget_bias=1.0)

    # Get lstm cell output
    outputs, states = rnn.rnn(lstm_cell, x, dtype=tf.float32)

    weights = tf.Variable(tf.random_normal([n_hidden, n_out], stddev=0.01))
    biases = tf.Variable(tf.random_normal([n_out], stddev=0.01))
    # Linear activation, using rnn inner loop last output
    return tf.matmul(outputs[-1], weights) + biases

pred = RNN(x)

# Compute the cost for this batch of data via L2
cost = tf.reduce_mean(tf.pow(pred-y, 2))    
# compute updates to parameters in order to minimize cost
train_op = tf.train.RMSPropOptimizer(0.005, 0.2).minimize(cost)

In [ ]:
#Defining some hyper-params
num_iter = 50000
print_step = 100
validation_x, validation_y = gen_data(min_sequence_len, max_sequence_len, batch_size)        
validation = tf.reduce_mean(pred-y)
with tf.Session() as sess:
    tf.initialize_all_variables().run()     # Initialize all variables in the model.
    for i in range(num_iter):
        batch_x, batch_y = gen_data(min_sequence_len, max_sequence_len, batch_size)        
#         batch_y = np.reshape(batch_y, [batch_size, 1])        
        sess.run([train_op], feed_dict={x:batch_x, y:batch_y})
        if i % print_step == 0:
            print sess.run([validation], feed_dict={x:validation_x, y:validation_y})

        

[-0.99700838]
[-0.11385787]
[0.038903829]
[-0.13637443]
[-0.065999694]
[0.050550047]
[-0.017763291]
[-0.071639508]
[0.1066631]
[-0.11529452]
[-0.089304686]
[0.061037645]
[-0.022104749]
